# Imports

In [69]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.fire_data import *
from src.data.read_data import *
from src.gen_functions import *
from src.features.dataset import Dataset
from src.features.build_features import *
from src.models.train_model import *
import seaborn as sns
output_notebook()
# set font size 
plt.rcParams.update({'font.size': 16})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [2]:
b_folder='../data/pm25/'
a4th_folder='../data/air4thai_hourly/'
cm_folder ='../data/cm_proc/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'
w_folder = '../data/weather_cities/'

# Dataset Object

In [39]:
data = Dataset('Chiang Mai')
# build data from scratch 
data.build_all_data(build_fire=True,build_holiday=False)
data.save_()

save file ../data/chiang_mai/35t.csv
save file ../data/chiang_mai/36t.csv
Averaging data from 3 stations


  0%|                                                                                                              | 0/22 [00:00<?, ?it/s]

Loading all hotspots data. This might take sometimes


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [14:50<00:00, 40.46s/it]


before drop (3909154, 18)
after drop (3786610, 10)


In [116]:
data = Dataset('Chiang Mai')
# load raw data 
data.load_()
# build the first dataset 
data.feature_no_fire()
# use default fire feature
data.merge_fire()
data.pollutant = 'PM2.5'

data no fire has shape (77746, 15)
use defaul fire feature


# Optimize 1: Remove Low Importance Feature

In [126]:
# split the data into 4 set
data.split_data(split_ratio=[0.4, 0.2, 0.2, 0.2])
xtrn, ytrn, x_cols = data.get_data_matrix(use_index=data.split_list[0])
xval, yval, _ = data.get_data_matrix(use_index=data.split_list[1])

In [45]:
%%time 
model = do_rf_search(xtrn,ytrn)
print(cal_scores(ytrn, model.predict(xtrn), header_str ='trn_'))

{'n_estimators': 20, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 17, 'max_depth': None, 'bootstrap': True} 0.6731723739152458
{'trn_r2_score': 0.9160096161908925, 'trn_mean_squared_error': 59.29360208146839}
Wall time: 6min 40s


In [127]:
model = RandomForestRegressor(**{'n_estimators': 20, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 17, 'max_depth': None, 'bootstrap': True})
model.fit(xtrn,ytrn)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=17, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=20, min_weight_fraction_leaf=0.0,
                      n_estimators=20, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [131]:
importances = model.feature_importances_
feat_imp = pd.DataFrame(importances, index=x_cols, columns=['importance']) 
feat_imp = feat_imp.sort_values('importance').reset_index()
to_drop = feat_imp['index']

# try droping feature without fires
to_drop = [a for a in to_drop if 'fire' not in a]
print(to_drop)
model, x_cols = reduce_cols(dataset=data,x_cols=x_cols,to_drop=to_drop,model=model,trn_i=0, val_i=1)
print(x_cols)

['wind_CALM', 'is_weekend', 'is_holiday', 'wind_E', 'wind_S', 'wind_N', 'wind_W', 'is_rain', 'day_of_week', 'Wind Speed(kmph)', 'time_of_day', 'Humidity(%)', 'Temperature(C)', 'PM2.5_lag_1']
old score 0.9360231916937746 new score 0.9379171313863407
drop is_weekend
old score 0.939044671648231 new score 0.9393200559018542
drop is_holiday
old score 0.9352911711687815 new score 0.9385996233464963
drop wind_E
old score 0.9357992103083583 new score 0.9411332256430496
drop wind_N
old score 0.9385894919742103 new score 0.9388530281835471
drop is_rain
old score 0.9374978042495471 new score 0.9391680343816142
drop day_of_week
old score 0.938568713870941 new score 0.9388217310205224
drop Wind Speed(kmph)
old score 0.9377787528349604 new score 0.9380151555784816
drop time_of_day
old score 0.9357998727890539 new score 0.9375993541678994
drop Humidity(%)
use columns Index(['Temperature(C)', 'PM2.5_lag_1', 'wind_CALM', 'wind_S', 'wind_W',
       'fire_0_100', 'fire_100_400', 'fire_400_700', 'fire_700

In [132]:
data.x_cols = x_cols

In [137]:
data.fire_dict  = {'w_speed': 8, 'shift': -18, 'roll': 114}

# Optimize 2: optimize fire feature using skopt

In [133]:
# reduce the number of split
data.split_data(split_ratio=[0.6, 0.2, 0.2])

In [134]:
trn_index = data.split_list[0]
val_index = data.split_list[1]

In [177]:
%%time 
sk_op_fire(data, model, trn_index, val_index)

old score 0.9348477659311422 fire dict {'w_speed': 4, 'shift': -24, 'roll': 108}
optimizing fire parameter. This will take about 15 mins


c:\users\benny\pyenv\geo\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\users\benny\pyenv\geo\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\users\benny\pyenv\geo\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\users\benny\pyenv\geo\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\users\benny\pyenv\geo\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\users\benny\

r2 score for the best fire parameters 0.9433646411755909
Wall time: 17min 38s


{'w_speed': 11.5905021535098, 'shift': -44, 'roll': 24}

# Try Optimize Fire Before Low Importance Feature

In [182]:
data = Dataset('Chiang Mai')
# load raw data 
data.load_()
# build the first dataset 
data.feature_no_fire()
# use default fire feature
data.merge_fire()
data.pollutant = 'PM2.5'

data no fire has shape (77746, 15)
use default fire feature


In [186]:
data.merge_fire()


use default fire feature


In [183]:
# split the data into 4 set
data.split_data(split_ratio=[0.4, 0.2, 0.2, 0.2])
xtrn, ytrn, x_cols = data.get_data_matrix(use_index=data.split_list[0])
xval, yval, _ = data.get_data_matrix(use_index=data.split_list[1])

# optimize for best rf 

model = do_rf_search(xtrn,ytrn)
print(cal_scores(ytrn, model.predict(xtrn), header_str ='trn_'))

{'n_estimators': 20, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 17, 'max_depth': None, 'bootstrap': True} 0.6731723739152458
{'trn_r2_score': 0.9160096161908925, 'trn_mean_squared_error': 59.29360208146839}


In [192]:
xtrn, ytrn, x_cols = data.get_data_matrix(use_index=data.split_list[0])
xval, yval, _ = data.get_data_matrix(use_index=data.split_list[1])
data.x_cols = x_cols
model = RandomForestRegressor(**{'n_estimators': 20, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 17, 'max_depth': None, 'bootstrap': True} )

In [ ]:
%%time
##optimize fire feature 
sk_op_fire(data, model, data.split_list[0], data.split_list[1])

old score 0.933386542838072 fire dict {'w_speed': 4, 'shift': -24, 'roll': 108}
optimizing fire parameter. This will take about 15 mins


In [185]:
importances = model.feature_importances_
feat_imp = pd.DataFrame(importances, index=x_cols, columns=['importance']) 
feat_imp = feat_imp.sort_values('importance').reset_index()
to_drop = feat_imp['index']

# try droping feature without fires
to_drop = [a for a in to_drop if 'fire' not in a]
print(to_drop)
model, x_cols = reduce_cols(dataset=data,x_cols=x_cols,to_drop=to_drop,model=model,trn_i=0, val_i=1)
print(x_cols)

['wind_CALM', 'is_weekend', 'is_holiday', 'wind_S', 'wind_E', 'wind_N', 'wind_W', 'is_rain', 'day_of_week', 'Wind Speed(kmph)', 'time_of_day', 'Humidity(%)', 'Temperature(C)', 'PM2.5_lag_1']
old score 0.9362703360708334 new score 0.9373914966196282
drop wind_CALM
old score 0.9373914966196282 new score 0.937780124902004
drop is_weekend
old score 0.9349077995939692 new score 0.9378110257561949
drop wind_S
old score 0.9378110257561949 new score 0.937886419320934
drop wind_E
old score 0.9362746598178632 new score 0.937606339306785
drop wind_W
old score 0.9360636417574594 new score 0.9371547802281873
drop day_of_week
use columns Index(['Temperature(C)', 'Humidity(%)', 'Wind Speed(kmph)', 'PM2.5_lag_1',
       'wind_N', 'is_rain', 'is_holiday', 'time_of_day', 'fire_0_100',
       'fire_100_400', 'fire_400_700', 'fire_700_1000'],
      dtype='object')
r2 score 0.9386234994658706
Index(['Temperature(C)', 'Humidity(%)', 'Wind Speed(kmph)', 'PM2.5_lag_1',
       'wind_N', 'is_rain', 'is_holiday'